<center><h1>Vision Transformer</h1> </center>

<center><p><a href="http://arxiv.org/abs/2010.11929">An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale</a></p></center>

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2021-01-26_at_9.43.31_PM_uI4jjMq.png" width="600"/>

[Code: https://github.com/huggingface/pytorch-image-models](https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py)


In [1]:
import collections.abc
from itertools import repeat
from functools import partial

import torch
import torch.nn as nn
from torch.nn.init import trunc_normal_

# Model Architecture

## Embeddings

### Patch Embedding

In [2]:
class PatchEmbedding(nn.Module):
    def __init__(
            self,
            img_size=224,
            patch_size=16,
            in_channels=3,
            embed_dim=768,
    ):
        """
        Patch Embedding.
        :param img_size: input image size. Default: 224
        :param patch_size: patch size. Default: 16
        :param in_channels: number of image input channels. Default: 3
        :param embed_dim: transformer embedding dimension. Default: 768
        """
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = (
            img_size[0] // patch_size[0],
            img_size[1] // patch_size[1],
        )
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.proj = nn.Conv2d(in_channels, embed_dim, patch_size, patch_size)

    def forward(self, x):
        """
        Patch Embedding.
        :param x: (batch_size, in_channels, height, width)
        :return: (batch_size, num_patches, embed_dim)
        """
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match " \
            f"model ({self.img_size[0]}*{self.img_size[1]})."

        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

### Position Embedding

In [3]:
class PositionEmbedding(nn.Module):
    def __init__(self, seq_length=197, embed_dim=768, pos_drop_rate=0.):
        """
        Position Embedding.
        :param seq_length: transformer’s sequence length. Default: 197 = 196 + 1
        :param embed_dim: transformer embedding dimension. Default: 768
        :param pos_drop_rate: position embedding dropout rate. Default: 0
        """
        super().__init__()
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, seq_length, embed_dim).float() * .02
        )
        self.pos_drop = nn.Dropout(pos_drop_rate)
        self._init_weights()

    def forward(self, x):
        """
        Position Embedding.
        :param x: (batch_size, num_patches, embed_dim)
        :return: (batch_size, num_patches + 1, embed_dim)
        """
        x = torch.cat((self.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        return x

    def _init_weights(self):
        trunc_normal_(self.pos_embed, std=.02)
        nn.init.normal_(self.cls_token, std=1e-6)

## Sublayers

### Multi-Head Self-Attention (MSA)

In [4]:
class Attention(nn.Module):
    def __init__(
            self,
            dim=768,
            num_heads=12,
            qkv_bias=False,
            qk_norm=False,
            attn_drop_rate=0.,
            proj_drop_rate=0.,
            norm_layer=nn.LayerNorm,
    ):
        """
        Multi-head Self-Attention.
        :param dim: transformer embedding dimension. Default: 768
        :param num_heads: number of attention heads. Default: 12
        :param qkv_bias: enable bias for qkv projections if True. Default: False
        :param qk_norm: norm qk before attention if True. Default: False
        :param attn_drop_rate: attention dropout rate. Default: 0
        :param proj_drop_rate: projection layer dropout rate. Default: 0
        :param norm_layer: normalization layer. Default: nn.LayerNorm
        """
        super(Attention, self).__init__()
        assert dim % num_heads == 0, 'dim should be divisible by num_heads'
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.q_norm = norm_layer(self.head_dim) if qk_norm else nn.Identity()
        self.k_norm = norm_layer(self.head_dim) if qk_norm else nn.Identity()
        self.attn_drop = nn.Dropout(attn_drop_rate)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop_rate)

    def forward(self, x):
        """
        Multi-head Self-Attention.
        :param x: (batch_size, seq_length, dim)
        :return: (batch_size, seq_length, dim)
        """
        B, N, C = x.shape

        qkv = (
            self.qkv(x)
            .reshape(B, N, 3, self.num_heads, self.head_dim)
            .permute(2, 0, 3, 1, 4)
        )
        q, k, v = qkv.unbind(0)
        q, k = self.q_norm(q), self.k_norm(k)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x = attn @ v

        x = x.transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

### Multilayer Perceptron (MLP)

In [5]:
class Mlp(nn.Module):
    def __init__(
            self,
            in_features=768,
            hidden_features=None,
            out_features=None,
            drop_rate=0.,
    ):
        """
        Multilayer Perceptron.
        :param in_features: input dimension. Default: 768
        :param hidden_features: hidden dimension. Default: in_features if None
        :param out_features: output dimension. Default: in_features if None
        :param drop_rate: dropout rate. Default: 0
        """
        super().__init__()
        hidden_features = hidden_features or in_features
        out_features = out_features or in_features
        drop_probs = to_2tuple(drop_rate)
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.drop1 = nn.Dropout(drop_probs[0])
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop2 = nn.Dropout(drop_probs[1])

    def forward(self, x):
        """
        Multilayer Perceptron.
        :param x: (batch_size, seq_length, in_features)
        :return: (batch_size, seq_length, out_features)
        """
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.drop2(x)
        return x

## Transformer Block

In [6]:
class Block(nn.Module):
    def __init__(
            self,
            dim,
            num_heads,
            mlp_ratio=4.,
            qkv_bias=False,
            qk_norm=False,
            attn_drop_rate=0.,
            proj_drop_rate=0.,
            drop_path_rate=0.,
            norm_layer=nn.LayerNorm,
    ):
        """
        Transformer Block.
        :param dim: transformer embedding dimension. Default: 768
        :param num_heads: number of attention heads. Default: 12
        :param mlp_ratio: ratio of mlp hidden dim to embedding dim. Default: 4
        :param qkv_bias: enable bias for qkv projections if True. Default: False
        :param qk_norm: norm qk before attention if True. Default: False
        :param proj_drop_rate: projection layer dropout rate. Default: 0
        :param attn_drop_rate: attention dropout rate. Default: 0
        :param drop_path_rate: stochastic depth rate. Default: 0
        :param norm_layer: normalization layer. Default: nn.LayerNorm
        """
        super(Block, self).__init__()

        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim=dim,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_norm=qk_norm,
            attn_drop_rate=attn_drop_rate,
            proj_drop_rate=proj_drop_rate,
            norm_layer=norm_layer,
        )
        self.drop_path1 = DropPath(drop_path_rate) \
            if drop_path_rate > 0. else nn.Identity()

        self.norm2 = norm_layer(dim)
        self.mlp = Mlp(
            in_features=dim,
            hidden_features=int(dim * mlp_ratio),
            drop_rate=proj_drop_rate,
        )
        self.drop_path2 = DropPath(drop_path_rate) \
            if drop_path_rate > 0. else nn.Identity()

    def forward(self, x):
        """
        Transformer Block.
        :param x: (batch_size, seq_length, embed_dim)
        :return: (batch_size, seq_length, embed_dim)
        """
        x = x + self.drop_path1(self.attn(self.norm1(x)))
        x = x + self.drop_path2(self.mlp(self.norm2(x)))
        return x

## Vision Transformer (ViT)

In [7]:
class VisionTransformer(nn.Module):
    def __init__(
            self,
            img_size: int = 224,
            patch_size: int = 16,
            in_channels: int = 3,
            num_classes: int = 1000,
            embed_dim: int = 768,
            depth: int = 12,
            num_heads: int = 12,
            mlp_ratio: float = 4.,
            qkv_bias: bool = True,
            qk_norm: bool = None,
            drop_rate: float = 0.,
            pos_drop_rate: float = 0.,
            patch_drop_rate: float = 0.,
            proj_drop_rate: float = 0.,
            attn_drop_rate: float = 0.,
            drop_path_rate: float = 0.,
    ):
        """
        Vision Transformer.
        :param img_size: input image size. Default: 224
        :param patch_size: patch size. Default: 16
        :param in_channels: number of image input channels. Default: 3
        :param num_classes: number of classes for classification head. Default: 1000
        :param embed_dim: transformer embedding dimension. Default: 768
        :param depth: depth of transformer. Default: 12
        :param num_heads: number of attention heads. Default: 12
        :param mlp_ratio: ratio of mlp hidden dim to embedding dim. Default: 4
        :param qkv_bias: enable bias for qkv projections if True. Default: False
        :param qk_norm: norm qk before attention if True. Default: False
        :param drop_rate: classifier head dropout rate. Default: 0
        :param pos_drop_rate: position embedding dropout rate. Default: 0
        :param patch_drop_rate: patch dropout rate. Default: 0
        :param proj_drop_rate: projection layer dropout rate. Default: 0
        :param attn_drop_rate: attention dropout rate. Default: 0
        :param drop_path_rate: stochastic depth rate. Default: 0
        """
        super(VisionTransformer, self).__init__()
        norm_layer = partial(nn.LayerNorm, eps=1e-6)

        self.patch_embed = PatchEmbedding(
            img_size=img_size,
            patch_size=patch_size,
            in_channels=in_channels,
            embed_dim=embed_dim
        )
        self.pos_embed = PositionEmbedding(
            seq_length=self.patch_embed.num_patches + 1,
            embed_dim=embed_dim,
            pos_drop_rate=pos_drop_rate
        )

        self.patch_drop = PatchDropout(patch_drop_rate) \
            if patch_drop_rate > 0 else nn.Identity()

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]

        self.blocks = nn.Sequential(
            *[
                Block(
                    dim=embed_dim,
                    num_heads=num_heads,
                    mlp_ratio=mlp_ratio,
                    qkv_bias=qkv_bias,
                    qk_norm=qk_norm,
                    attn_drop_rate=attn_drop_rate,
                    proj_drop_rate=proj_drop_rate,
                    drop_path_rate=dpr[i],
                    norm_layer=norm_layer,
                )
                for i in range(depth)
            ]
        )
        self.norm = norm_layer(embed_dim)

        self.head_drop = nn.Dropout(drop_rate)
        self.head = nn.Linear(embed_dim, num_classes)

        self._init_weights()

    def forward(self, x):
        """
        Vision Transformer.
        :param x: (batch_size, in_channels, height, width)
        :return: (batch_size, num_classes)
        """
        x = self._forward_features(x)
        x = self._forward_head(x)
        return x

    def _forward_features(self, x):
        """
        ViT Backbone.
        :param x: (batch_size, in_channels, height, width)
        :return: (batch_size, num_patches + 1, embed_dim)
        """
        x = self.patch_embed(x)  # (batch_size, num_patches, embed_dim)
        x = self.pos_embed(x)  # (batch_size, num_patches + 1, embed_dim)
        x = self.blocks(x)  # (batch_size, num_patches + 1, embed_dim)
        x = self.norm(x)  # (batch_size, num_patches + 1, embed_dim)
        return x

    def _forward_head(self, x):
        """
        ViT Classification Head.
        :param x: (batch_size, num_patches + 1, embed_dim)
        :return: (batch_size, num_classes)
        """
        x = x[:, 0, :]  # (batch_size, embed_dim)
        x = self.head_drop(x)  # (batch_size, embed_dim)
        x = self.head(x)  # (batch_size, num_classes)
        return x

    def _init_weights(self):
        self.apply(_init_vit_weights)

# Utils

## ViT init weights

In [8]:
def _init_vit_weights(m):
    """
    ViT weight initialization.
    :param m: module
    """
    if isinstance(m, nn.Linear):
        nn.init.trunc_normal_(m.weight, std=.01)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode="fan_out")
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LayerNorm):
        nn.init.zeros_(m.bias)
        nn.init.ones_(m.weight)

## Drop paths

Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks). This is the same as the DropConnect impl I created for EfficientNet, etc. networks, however, the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...

See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ...

I've opted for changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use 'survival rate' as the argument.

In [9]:
class DropPath(nn.Module):
    """
    Drop paths (Stochastic Depth) per sample when applied
    in main path of residual blocks.
    """

    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Drop paths (Stochastic Depth) per sample when applied
    in main path of residual blocks.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    # work with diff dim tensors, not just 2D ConvNets
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output

## Patch Dropout

Scaling Language-Image Pre-training via Masking: https://arxiv.org/abs/2212.00794

In [10]:
class PatchDropout(nn.Module):
    """
    https://arxiv.org/abs/2212.00794
    """

    def __init__(self, prob=0.5):
        super().__init__()
        self.prob = prob

    def forward(self, x):
        if not self.training or self.prob == 0.:
            return x

        prefix_tokens, x = x[:, :1], x[:, 1:]

        B = x.shape[0]
        L = x.shape[1]
        num_keep = max(1, int(L * (1. - self.prob)))
        keep_indices = torch.argsort(
            torch.randn(B, L, device=x.device), dim=-1
        )[:, :num_keep]
        x = x.gather(
            1, keep_indices.unsqueeze(-1).expand((-1, -1) + x.shape[2:])
        )

        x = torch.cat((prefix_tokens, x), dim=1)

        return x

## _ntuple

In [11]:
# From PyTorch internals
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return tuple(x)
        return tuple(repeat(x, n))

    return parse


to_2tuple = _ntuple(2)

# Summary

## Data

In [12]:
device = 'cpu' if torch.cuda.is_available() else 'cpu'

in_channel = 3
image_size = 224
image_size_384 = 384
batch_size = 2
num_classes = 1000
num_classes_in21k = 21843

data = torch.randn((batch_size, in_channel, image_size, image_size)).to(device)
data_384 = torch.randn((batch_size, in_channel, image_size_384, image_size_384)).to(device)

## ViT-B/16

In [13]:
from torchkeras import summary

net = VisionTransformer(
    img_size=224,
    patch_size=16,
    embed_dim=768,
    depth=12,
    num_heads=12,
    num_classes=num_classes
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                           [-1, 768, 14, 14]              590,592
Dropout-2                             [-1, 197, 768]                    0
LayerNorm-3                           [-1, 197, 768]                1,536
Linear-4                             [-1, 197, 2304]            1,771,776
Identity-5                         [-1, 12, 197, 64]                    0
Identity-6                         [-1, 12, 197, 64]                    0
Dropout-7                         [-1, 12, 197, 197]                    0
Linear-8                              [-1, 197, 768]              590,592
Dropout-9                             [-1, 197, 768]                    0
Identity-10                           [-1, 197, 768]                    0
LayerNorm-11                          [-1, 197, 768]                1,536
Linear-12                            

## ViT-B/32

In [14]:
net = VisionTransformer(
    img_size=224,
    patch_size=32,
    embed_dim=768,
    depth=12,
    num_heads=12,
    num_classes=num_classes
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                             [-1, 768, 7, 7]            2,360,064
Dropout-2                              [-1, 50, 768]                    0
LayerNorm-3                            [-1, 50, 768]                1,536
Linear-4                              [-1, 50, 2304]            1,771,776
Identity-5                          [-1, 12, 50, 64]                    0
Identity-6                          [-1, 12, 50, 64]                    0
Dropout-7                           [-1, 12, 50, 50]                    0
Linear-8                               [-1, 50, 768]              590,592
Dropout-9                              [-1, 50, 768]                    0
Identity-10                            [-1, 50, 768]                    0
LayerNorm-11                           [-1, 50, 768]                1,536
Linear-12                            

## ViT-L/16

In [15]:
net = VisionTransformer(
    img_size=224,
    patch_size=16,
    embed_dim=1024,
    depth=24,
    num_heads=16,
    num_classes=num_classes
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 1024, 14, 14]              787,456
Dropout-2                            [-1, 197, 1024]                    0
LayerNorm-3                          [-1, 197, 1024]                2,048
Linear-4                             [-1, 197, 3072]            3,148,800
Identity-5                         [-1, 16, 197, 64]                    0
Identity-6                         [-1, 16, 197, 64]                    0
Dropout-7                         [-1, 16, 197, 197]                    0
Linear-8                             [-1, 197, 1024]            1,049,600
Dropout-9                            [-1, 197, 1024]                    0
Identity-10                          [-1, 197, 1024]                    0
LayerNorm-11                         [-1, 197, 1024]                2,048
Linear-12                            

## ViT-L/32

In [16]:
net = VisionTransformer(
    img_size=224,
    patch_size=32,
    embed_dim=1024,
    depth=24,
    num_heads=16,
    num_classes=num_classes
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                            [-1, 1024, 7, 7]            3,146,752
Dropout-2                             [-1, 50, 1024]                    0
LayerNorm-3                           [-1, 50, 1024]                2,048
Linear-4                              [-1, 50, 3072]            3,148,800
Identity-5                          [-1, 16, 50, 64]                    0
Identity-6                          [-1, 16, 50, 64]                    0
Dropout-7                           [-1, 16, 50, 50]                    0
Linear-8                              [-1, 50, 1024]            1,049,600
Dropout-9                             [-1, 50, 1024]                    0
Identity-10                           [-1, 50, 1024]                    0
LayerNorm-11                          [-1, 50, 1024]                2,048
Linear-12                            

## ViT-H/14

In [17]:
net = VisionTransformer(
    img_size=224,
    patch_size=14,
    embed_dim=1280,
    depth=32,
    num_heads=16,
    num_classes=num_classes
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 1280, 16, 16]              753,920
Dropout-2                            [-1, 257, 1280]                    0
LayerNorm-3                          [-1, 257, 1280]                2,560
Linear-4                             [-1, 257, 3840]            4,919,040
Identity-5                         [-1, 16, 257, 80]                    0
Identity-6                         [-1, 16, 257, 80]                    0
Dropout-7                         [-1, 16, 257, 257]                    0
Linear-8                             [-1, 257, 1280]            1,639,680
Dropout-9                            [-1, 257, 1280]                    0
Identity-10                          [-1, 257, 1280]                    0
LayerNorm-11                         [-1, 257, 1280]                2,560
Linear-12                            

## ViT-H/14_in21k_384

In [18]:
net = VisionTransformer(
    img_size=384,
    patch_size=14,
    embed_dim=1280,
    depth=32,
    num_heads=16,
    num_classes=num_classes_in21k
).to(device)

summary(net, input_data=data_384)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 1280, 27, 27]              753,920
Dropout-2                            [-1, 730, 1280]                    0
LayerNorm-3                          [-1, 730, 1280]                2,560
Linear-4                             [-1, 730, 3840]            4,919,040
Identity-5                         [-1, 16, 730, 80]                    0
Identity-6                         [-1, 16, 730, 80]                    0
Dropout-7                         [-1, 16, 730, 730]                    0
Linear-8                             [-1, 730, 1280]            1,639,680
Dropout-9                            [-1, 730, 1280]                    0
Identity-10                          [-1, 730, 1280]                    0
LayerNorm-11                         [-1, 730, 1280]                2,560
Linear-12                            